# 🔌 Model Context Protocol (MCP) with Strands Agents

Learn how to integrate Model Context Protocol servers with Strands Agents.

## What is MCP?

Model Context Protocol (MCP) is an open standard that enables AI applications to connect with external data sources and tools.

### Benefits:
- 🔄 Standardized tool integration
- 🌐 Connect to external services
- 🔧 Reusable tool servers
- 🚀 Easy deployment

![MCP Architecture](image/mcp.png)

In [1]:
import boto3
from strands import Agent
from strands.models import BedrockModel
from strands.tools import tool

print("✅ Imports successful!")

✅ Imports successful!


## Understanding MCP Architecture

MCP consists of:
1. **MCP Server**: Exposes tools and resources
2. **MCP Client**: Connects to servers and uses tools
3. **Protocol**: Standardized communication format

Strands Agents can act as an MCP client to use tools from MCP servers.

## MCP Server Transport Types

Strands Agents supports **4 ways to create MCP servers**, each with different transport mechanisms:

### 1. 📟 Standard I/O (stdio)
**Communication**: Process stdin/stdout streams

**Use Cases:**
- Command-line tools
- Local development
- Process-to-process communication

**Example:**
```python
# Server
mcp.run(transport="stdio")

# Client
from mcp.client.stdio import stdio_client
client = MCPClient(lambda: stdio_client("python", "server.py"))
```

**Pros:** Simple, no network setup, secure (local only)

**Cons:** Single process, not suitable for distributed systems

---

### 2. 🌊 Streamable HTTP
**Communication**: HTTP with streaming responses

**Use Cases:**
- Real-time data streaming
- Long-running operations
- Progressive results

**Example:**
```python
# Server
mcp.run(transport="http", streaming=True)

# Client
from mcp.client.http import http_client
client = MCPClient(lambda: http_client("http://localhost:8000"))
```

**Pros:** Real-time updates, efficient for large responses

**Cons:** More complex than SSE, requires streaming support

---

### 3. 📡 Server-Sent Events (SSE)
**Communication**: HTTP with server-sent events

**Use Cases:**
- Web-based integrations
- Remote tool servers
- Production deployments
- **This is what we use in our examples!**

**Example:**
```python
# Server
mcp.run(transport="sse")  # Default port: 8000

# Client
from mcp.client.sse import sse_client
client = MCPClient(lambda: sse_client("http://localhost:8000/sse"))
```

**Pros:** HTTP-based, firewall-friendly, easy to deploy

**Cons:** One-way communication (server to client)

---

### 4. 🔧 Custom Transport with MCP Client
**Communication**: Your own transport implementation

**Use Cases:**
- Custom protocols (WebSocket, gRPC, etc.)
- Special security requirements
- Integration with existing systems

**Example:**
```python
# Implement custom transport
class CustomTransport:
    async def connect(self): ...
    async def send(self, message): ...
    async def receive(self): ...

# Use with MCPClient
client = MCPClient(lambda: CustomTransport())
```

**Pros:** Maximum flexibility, custom protocols

**Cons:** Requires custom implementation

---

### Transport Comparison

| Transport | Network | Streaming | Complexity | Best For |
|-----------|---------|-----------|------------|----------|
| **stdio** | ❌ Local | ❌ | Low | CLI tools, local dev |
| **HTTP** | ✅ Remote | ✅ | Medium | Real-time streaming |
| **SSE** | ✅ Remote | ✅ | Low | Web apps, production |
| **Custom** | ✅ Flexible | ✅ | High | Special requirements |

### 📝 In This Notebook

We'll demonstrate **two transport types**:
1. **SSE (Server-Sent Events)** - For our HTTP-based MCP server
2. **stdio (Standard I/O)** - For command-line usage (optional)

Both examples use the same tools, just different transport mechanisms!

## Creating MCP Servers from Custom Tools

Let's convert the custom tools from notebook 02 into MCP servers using **two different transports**!

The servers will expose:
- 🧮 **calculator**: Basic math operations
- ⏰ **get_current_time**: Current date/time
- 🎥 **video_reader_local**: Video analysis with Bedrock

### Example 1: SSE Transport (Server-Sent Events)

**Transport Type:** 📡 Server-Sent Events (SSE)

**Best for:** Web-based integrations, remote access, production deployments

We'll create `mcp_custom_tools_server.py` with FastMCP using SSE transport:

In [2]:
# Create the MCP server file
mcp_server_code = '''#!/usr/bin/env python3
"""
MCP Server for Custom Tools
This server exposes calculator, time, and video analysis tools via Model Context Protocol.

Built with FastMCP from Strands Agents ecosystem.
"""
import os
from datetime import datetime
from mcp.server import FastMCP

# Import video reader tool
from video_reader_local import video_reader_local

# Create FastMCP server
mcp = FastMCP("Custom Tools Server")


@mcp.tool(description="Performs basic mathematical operations (add, subtract, multiply, divide)")
def calculator(operation: str, a: float, b: float) -> str:
    """
    Calculator tool for basic math operations.
    
    Args:
        operation: The operation to perform (add, subtract, multiply, divide)
        a: First number
        b: Second number
    
    Returns:
        The result of the operation as a string
    """
    operations = {
        "add": a + b,
        "subtract": a - b,
        "multiply": a * b,
        "divide": a / b if b != 0 else "Error: Division by zero"
    }
    
    result = operations.get(operation, "Invalid operation")
    return str(result)


@mcp.tool(description="Gets the current date and time in UTC timezone")
def get_current_time(timezone: str = "UTC") -> str:
    """
    Get current date and time.
    
    Args:
        timezone: Timezone (currently only UTC supported)
    
    Returns:
        Current date and time as a formatted string
    """
    now = datetime.now()
    return f"Current time ({timezone}): {now.strftime('%Y-%m-%d %H:%M:%S')}"


@mcp.tool(description="""Analyze video content using AWS Bedrock's multimodal capabilities.

IMPORTANT LIMITATIONS:
- Only 1 video per request
- No audio analysis (visual only)
- Cannot identify people
- Maximum file size: ~20MB
- Supported formats: mp4, mov, avi, mkv, webm
""")
def analyze_video(
    video_path: str,
    text_prompt: str = "Describe what you see in this video",
    model_id: str = "us.amazon.nova-pro-v1:0",
    region: str = "us-west-2"
) -> str:
    """
    Analyze video content using AWS Bedrock.
    
    Args:
        video_path: Path to local video file
        text_prompt: Question or instruction for analyzing the video
        model_id: Bedrock model ID to use
        region: AWS region for Bedrock
    
    Returns:
        Video analysis results as a string
    """
    # Call the video reader tool
    result = video_reader_local(
        video_path=video_path,
        text_prompt=text_prompt,
        model_id=model_id,
        region=region
    )
    
    # Extract text from result
    if result["status"] == "success":
        return result["content"][0]["text"]
    else:
        return result["content"][0]["text"]


# Run the server
if __name__ == "__main__":
    # Default to SSE transport for easy HTTP access
    # Can also use "stdio" for command-line usage
    mcp.run(transport="sse")
'''

# Write the server file
with open('mcp_custom_tools_server.py', 'w') as f:
    f.write(mcp_server_code)

print("✅ MCP server file created: mcp_custom_tools_server.py")
print("\n📝 Key MCP Server Components (SSE Transport):")
print("="*80)
print("1. from mcp.server import FastMCP")
print("2. mcp = FastMCP('Custom Tools Server')")
print("3. @mcp.tool() - Decorator to define tools")
print("4. mcp.run(transport='sse') - Start SSE HTTP server")
print("="*80)
print("\n🌐 SSE Server will run at: http://localhost:8000/sse")
print("📡 Transport: Server-Sent Events (SSE)")
print("✅ Best for: Web integrations, remote access, production")

✅ MCP server file created: mcp_custom_tools_server.py

📝 Key MCP Server Components (SSE Transport):
1. from mcp.server import FastMCP
2. mcp = FastMCP('Custom Tools Server')
3. @mcp.tool() - Decorator to define tools
4. mcp.run(transport='sse') - Start SSE HTTP server

🌐 SSE Server will run at: http://localhost:8000/sse
📡 Transport: Server-Sent Events (SSE)
✅ Best for: Web integrations, remote access, production


### Example 2: stdio Transport (Standard I/O)

**Transport Type:** 📟 Standard I/O (stdio)

**Best for:** Command-line tools, local development, process-to-process communication

Let's create an alternative version using stdio transport:

In [ ]:
# Create stdio version of the MCP server
stdio_server_code = '''#!/usr/bin/env python3
"""
MCP Server for Custom Tools - stdio Transport
This server uses Standard I/O for process-to-process communication.

Built with FastMCP from Strands Agents ecosystem.
"""
import os
from datetime import datetime
from mcp.server import FastMCP

# Import video reader tool
from video_reader_local import video_reader_local

# Create FastMCP server
mcp = FastMCP("Custom Tools Server - stdio")


@mcp.tool(description="Performs basic mathematical operations (add, subtract, multiply, divide)")
def calculator(operation: str, a: float, b: float) -> str:
    """Calculator tool for basic math operations."""
    operations = {
        "add": a + b,
        "subtract": a - b,
        "multiply": a * b,
        "divide": a / b if b != 0 else "Error: Division by zero"
    }
    result = operations.get(operation, "Invalid operation")
    return str(result)


@mcp.tool(description="Gets the current date and time in UTC timezone")
def get_current_time(timezone: str = "UTC") -> str:
    """Get current date and time."""
    now = datetime.now()
    return f"Current time ({timezone}): {now.strftime('%Y-%m-%d %H:%M:%S')}"


@mcp.tool(description="Analyze video content using AWS Bedrock")
def analyze_video(
    video_path: str,
    text_prompt: str = "Describe what you see in this video",
    model_id: str = "us.amazon.nova-pro-v1:0",
    region: str = "us-west-2"
) -> str:
    """Analyze video content using AWS Bedrock."""
    result = video_reader_local(
        video_path=video_path,
        text_prompt=text_prompt,
        model_id=model_id,
        region=region
    )
    
    if result["status"] == "success":
        return result["content"][0]["text"]
    else:
        return result["content"][0]["text"]


# Run the server with stdio transport
if __name__ == "__main__":
    mcp.run(transport="stdio")  # Use stdio for command-line communication
'''

# Write the stdio server file
with open('mcp_custom_tools_server_stdio.py', 'w') as f:
    f.write(stdio_server_code)

print("✅ stdio MCP server file created: mcp_custom_tools_server_stdio.py")
print("\n📝 Key Differences (stdio Transport):")
print("="*80)
print("1. mcp.run(transport='stdio') - Use stdin/stdout")
print("2. No HTTP server - direct process communication")
print("3. Invoked as: python mcp_custom_tools_server_stdio.py")
print("="*80)
print("\n📟 Transport: Standard I/O (stdio)")
print("✅ Best for: CLI tools, local development, process-to-process")
print("\n💡 Both servers expose the same tools, just different transports!")

### Transport Comparison: SSE vs stdio

| Feature | SSE (mcp_custom_tools_server.py) | stdio (mcp_custom_tools_server_stdio.py) |
|---------|----------------------------------|------------------------------------------|
| **Transport** | HTTP with Server-Sent Events | Standard I/O streams |
| **Network** | ✅ Remote access | ❌ Local only |
| **URL** | http://localhost:8000/sse | N/A (process communication) |
| **Client** | `sse_client(url)` | `stdio_client(command, args)` |
| **Use Case** | Web apps, production | CLI tools, local dev |
| **Firewall** | May need configuration | No network involved |
| **Deployment** | Docker, cloud services | Local processes |

### Which Transport to Choose?

**Use SSE when:**
- Building web applications
- Need remote access to tools
- Deploying to cloud (AWS, Docker)
- Multiple clients need access

**Use stdio when:**
- Building CLI tools
- Local development only
- Process-to-process communication
- No network access needed

**For this notebook, we'll use SSE** as it's more suitable for production scenarios and web-based integrations.

## Using MCP Servers with Strands Agents

Now let's connect to our **SSE-based MCP server** and use it with Strands Agents.

### Step 1: Start the SSE MCP Server

Open a **separate terminal** and run:

```bash
python mcp_custom_tools_server.py
```

The server will start on `http://localhost:8000/sse` by default.

**Note:** If you want to use the stdio version instead, you would invoke it differently (see manual approach below).

### Step 2: Connect to SSE MCP Server

In [3]:
from mcp.client.sse import sse_client
from strands.tools.mcp import MCPClient

# Connect to our custom tools MCP server via SSE
mcp_client = MCPClient(
    lambda: sse_client("http://localhost:8000/sse")
)

print("✅ MCP Client created (SSE Transport)")
print("📡 Transport: Server-Sent Events (SSE)")
print("🌐 Connecting to: http://localhost:8000/sse")
print("⚠️  Make sure mcp_custom_tools_server.py is running in another terminal!")
print("\n💡 Alternative: For stdio transport, use:")
print("   from mcp.client.stdio import stdio_client")
print("   client = MCPClient(lambda: stdio_client('python', 'mcp_custom_tools_server_stdio.py'))")

✅ MCP Client created
📡 Connecting to: http://localhost:8000/sse
⚠️  Make sure the MCP server is running in another terminal!


### Step 3: Create Agent with MCP Tools

Strands Agents supports two approaches for MCP integration:

#### Option A: Managed Integration (Recommended - Experimental)
The agent automatically manages the MCP connection lifecycle:

In [4]:
# Setup Bedrock model
session = boto3.Session(region_name='us-west-2')
bedrock_model = BedrockModel(
    model_id="us.anthropic.claude-3-5-sonnet-20241022-v2:0",
    boto_session=session
)

# Create agent with MCP client directly (managed lifecycle)
mcp_agent = Agent(
    model=bedrock_model,
    tools=[mcp_client],  # Pass MCPClient directly!
    system_prompt="""You are a helpful assistant with access to:
    - Calculator for math operations
    - Time information
    - Video analysis capabilities
    """
)

print("✅ Agent created with MCP tools (managed mode)!")
print("📦 The agent will automatically connect and discover tools")

✅ Agent created with MCP tools (managed mode)!
📦 The agent will automatically connect and discover tools


#### Option B: Manual Context Management
For production use, you can manually manage the connection:

In [ ]:
# Manual approach with explicit context management
# Uncomment to use this approach instead:

# with mcp_client:
#     # Get tools from MCP server
#     mcp_tools = mcp_client.list_tools_sync()
#     
#     # Create agent with tools
#     mcp_agent = Agent(
#         model=bedrock_model,
#         tools=mcp_tools,
#         system_prompt="You are a helpful assistant..."
#     )
#     
#     # Use agent within context
#     response = mcp_agent("What is 10 + 20?")
#     print(response)

print("ℹ️  Manual approach commented out - using managed mode above")

### Step 4: Test MCP Tools

In [5]:
# Test 1: Calculator via MCP
print("=== 🧮 CALCULATOR TEST ===")
response = mcp_agent("What is 456 multiplied by 789?")
print(response)
print("\n" + "="*80 + "\n")

=== 🧮 CALCULATOR TEST ===
I'll help you multiply those numbers using the calculator function.
Tool #1: calculator
456 multiplied by 789 equals 359,784.456 multiplied by 789 equals 359,784.





In [6]:
# Test 2: Time via MCP
print("=== ⏰ TIME TEST ===")
response = mcp_agent("What time is it right now?")
print(response)
print("\n" + "="*80 + "\n")

=== ⏰ TIME TEST ===
I'll check the current time for you.
Tool #2: get_current_time
The current time in UTC is 15:54:24 on November 11, 2025.The current time in UTC is 15:54:24 on November 11, 2025.





In [7]:
# Test 3: Video analysis via MCP
print("=== 🎥 VIDEO ANALYSIS TEST ===")
response = mcp_agent(
    "Analyze the video at data-sample/moderation-video.mp4 and describe what you see"
)
print(response)
print("\n" + "="*80 + "\n")

=== 🎥 VIDEO ANALYSIS TEST ===
I'll analyze the video at the specified path using the video analysis tool.
Tool #3: analyze_video
Based on the analysis, the video shows a sequence of scenes involving smoking and alcohol:
1. First, there's a person on a couch with a beer bottle who opens it using a red object
2. The same person is then shown holding the beer bottle (with a red cap) and smoking a cigarette
3. The scene then changes to show a woman in a grassy field, holding and then smoking a cigarette

The video appears to contain adult content related to alcohol consumption and smoking.Based on the analysis, the video shows a sequence of scenes involving smoking and alcohol:
1. First, there's a person on a couch with a beer bottle who opens it using a red object
2. The same person is then shown holding the beer bottle (with a red cap) and smoking a cigarette
3. The scene then changes to show a woman in a grassy field, holding and then smoking a cigarette

The video appears to contain ad

In [8]:
# Test 4: Multiple tools in one request
print("=== 🔧 MULTI-TOOL TEST ===")
response = mcp_agent(
    "What time is it? Also calculate 123 + 456 for me."
)
print(response)
print("\n" + "="*80 + "\n")

=== 🔧 MULTI-TOOL TEST ===
I'll help you with both requests by using the time and calculator functions.
Tool #4: get_current_time

Tool #5: calculator
The current time is 15:54:49 UTC on November 11, 2025.
And 123 + 456 = 579.The current time is 15:54:49 UTC on November 11, 2025.
And 123 + 456 = 579.





## Test multimodal agents with MCP server

Let's see a practical example combining multiple MCP tools:

In [9]:
# Real-world scenario: Video content moderation with timestamp
print("=== 🎯 PRACTICAL EXAMPLE: Content Moderation ===")

response = mcp_agent("""
Please help me with content moderation:
1. First, tell me what time this analysis is being done
2. Then analyze the video at data-sample/moderation-video.mp4
3. If you see any concerning content, calculate a risk score from 0-100
""")

print(response)
print("\n" + "="*80)

=== 🎯 PRACTICAL EXAMPLE: Content Moderation ===
I'll help you with this content moderation analysis step by step.

1. First, let's get the current time:
Tool #6: get_current_time
2. Now, let's analyze the video:
Tool #7: analyze_video
3. Based on the analysis, there are concerning elements that warrant a risk score calculation. The video contains:
- Alcohol consumption (beer drinking)
- Tobacco use (cigarette smoking)

Let me calculate a risk score based on these elements. Since both alcohol and tobacco use are age-restricted activities but legal in many jurisdictions, I'll calculate a moderate risk score:
Tool #8: calculator
Summary Report:
- Analysis Time: 15:55:51 UTC on November 11, 2025
- Content: Video shows alcohol consumption and tobacco use
- Risk Score: 70/100 
  (Calculated by assigning 35 points each for alcohol and tobacco content, as these are age-restricted substances that require content warnings and moderation for younger audiences)

Recommendation: This content should

## Converting Strands Tools to MCP Servers

Here's how we converted the tools from notebook 02 to an MCP server using **FastMCP**:

### Original Strands Tool:

In [ ]:
# Original Strands tool (from notebook 02)
@tool
def calculator(operation: str, a: float, b: float) -> float:
    """Performs basic mathematical operations."""
    operations = {
        "add": a + b,
        "subtract": a - b,
        "multiply": a * b,
        "divide": a / b if b != 0 else "Error: Division by zero"
    }
    return operations.get(operation, "Invalid operation")

print("✅ Original Strands tool")

### Converted to MCP Server with FastMCP:

**Option 1: SSE Transport (HTTP-based)**
```python
# In mcp_custom_tools_server.py

from mcp.server import FastMCP

# Create FastMCP server
mcp = FastMCP("Custom Tools Server")

# Define tool with decorator
@mcp.tool(description="Performs basic mathematical operations")
def calculator(operation: str, a: float, b: float) -> str:
    """Calculator tool for basic math operations."""
    operations = {
        "add": a + b,
        "subtract": a - b,
        "multiply": a * b,
        "divide": a / b if b != 0 else "Error: Division by zero"
    }
    result = operations.get(operation, "Invalid operation")
    return str(result)

# Run with SSE transport
if __name__ == "__main__":
    mcp.run(transport="sse")  # 📡 SSE transport for HTTP access
```

**Option 2: stdio Transport (Process-based)**
```python
# In mcp_custom_tools_server_stdio.py

# Same tool definitions...

# Run with stdio transport
if __name__ == "__main__":
    mcp.run(transport="stdio")  # 📟 stdio transport for CLI
```

### Key Differences from Strands Tools:
1. **FastMCP Simplicity**: Much simpler than raw MCP - just use `@mcp.tool()` decorator
2. **Type Hints**: FastMCP automatically generates JSON schema from Python type hints
3. **Transport Flexibility**: Choose from 4 transport types (stdio, SSE, HTTP, custom)
4. **Process Isolation**: Runs as separate process (HTTP server or CLI)
5. **Same Logic**: The core tool logic remains identical!
6. **Reusability**: One server, multiple clients can connect

### Transport Selection Guide:

```python
# 📟 stdio - Local CLI tools
mcp.run(transport="stdio")

# 📡 SSE - Web apps, production (our choice)
mcp.run(transport="sse")

# 🌊 HTTP Streaming - Real-time data
mcp.run(transport="http", streaming=True)

# 🔧 Custom - Your own protocol
mcp.run(transport=CustomTransport())
```

## Benefits of MCP Integration

### 1. Tool Reusability
Create tools once, use them across different AI applications

### 2. Standardization
Follow industry standards for tool integration

### 3. Separation of Concerns
- Tools run as separate processes
- Better security and isolation
- Independent scaling

### 4. Ecosystem
Access to growing ecosystem of MCP servers

## MCP Calculator Example

![MCP Calculator Diagram](image/mcp_calculator_diagram.png)

The diagram shows how an agent interacts with an MCP calculator server:
1. Agent receives user request
2. Agent calls MCP server tool
3. MCP server processes calculation
4. Result returned to agent
5. Agent responds to user

## Deploying MCP Servers

MCP servers can be deployed:
- 🖥️ Locally for development
- 🐳 In containers (Docker)
- ☁️ On AWS Lambda, ECS, EC2, Agentcore Runtime, AgentCore Gateway.
- 🌐 As standalone services


## Cleanup

When using managed mode, the agent handles cleanup automatically.
For manual mode, close the MCP client:

In [ ]:
# Managed mode: No cleanup needed - agent handles it automatically
print("✅ Using managed mode - automatic cleanup")

# Manual mode: Uncomment if using manual context management
# mcp_client.close()
# print("✅ MCP client closed")

## Summary

In this notebook, you learned:

✅ What Model Context Protocol (MCP) is and why it matters

✅ How to convert Strands tools to MCP servers

✅ How to connect to MCP servers from Strands Agents

✅ How to use MCP tools in your agents

✅ Real-world examples of MCP integration


### Next Steps

Continue to the next notebook to learn about State & Sessions management!